<a href="https://colab.research.google.com/github/tguinot/difiq/blob/main/CRR_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pricing dans l'arbre de Cox-Ross-Rubinstein : options européennes vs. options américaines
-----

> **Antonin Chaix**

Le but de ce TP et de mettre en place l'arbre de Cox-Ross-Rubinstein, connu pour converger vers le modèle de Black & Scholes, afin de pricer quelques options :
* call et put européens (pour constater qu'on retombe sur les prix BS)
* call et put américains (pour constater qu'en l'absence de dividendes, suivant le signe du taux d'intérêt, le prix de l'un ou de l'autre coïncide avec celui de son cousin européen. On pourra également s'intéresser au cas avec dividendes.)
* call à barrière down & out (pour s'amuser un peu)

Pour plus de détails sur le modèle binomial et l'arbre de pricing associé on pourra se reporter à cette [page wikipédia](https://en.wikipedia.org/wiki/Binomial_options_pricing_model).

De notre côté, allons à l'essentiel : on définit les multiplicateurs $u$ et $d$ en fonction de la volatilité $\sigma$ afin de faire converger le modèle binomial multi-période vers Black-Scholes :

$$
u = e^{\sigma\sqrt{\Delta t}}\qquad d = e^{\sigma\sqrt{\Delta t}}
$$

et la probabilité risque-neutre :
$$
p = \frac{e^{(r-q)\Delta t}-d}{u-d}
$$

permettant de calculer à chaque noeud de l'arbre la valeur de l'option en fonction des valeurs de l'option aux deux noeuds associés au pas de temps suivant :

$$
V_{t,k} = e^{-r\Delta t}\left(pV_{t+\Delta t,k}+(1-p)V_{t+\Delta t,k+1}\right)
$$

Cette valeur est amenée à être modifiée pour les produits américains ou path-dependent. Par exemple pour un call américain, on introduit la stratégie d'exercice de la façon suivante à chaque noeud (exercice vs. conservation de l'option) :

$$
V_{t,k} = \max(S_{t,k}-K,e^{-r\Delta t}\left(pV_{t+\Delta t,k}+(1-p)V_{t+\Delta t,k+1}\right) )
$$

ou encore pour une option à barrière se désactivant si le sous-jacent descend en dessous d'une barrière $H$ :

$$
V_{t,k} = e^{-r\Delta t}\left(pV_{t+\Delta t,k}+(1-p)V_{t+\Delta t,k+1}\right) \mathbb{1}_{\{S_{t,k}>H\}}
$$


## Construction de l'arbre

On construit l'arbre avec les valeurs du sous-jacent dans un tableau de dimension `n_steps+1 x n_steps+1` où `n_steps` représente le nombre de pas de temps.

In [ ]:
import numpy as np
import math

# underlying params
S0 = 100
sigma = 0.2
r = 0.015
q = 0

# tree params
T = 1. # tree final date = options maturity
n_steps = 750

# Build tree for underlying
dt = T / n_steps
u = math.exp(sigma*math.sqrt(dt))
d = 1./u
p = (math.exp((r-q)*dt) - d) / (u - d)

S = np.zeros((n_steps+1, n_steps+1))
S[0,0] = S0

for i in range(1, n_steps+1) :
	S[:i,i] = S[:i,i-1] * u
	S[i,i] = S[i-1,i-1] * d


Pour bien se réprésenter ce qu'on fait, voilà à quoi ressemble l'arbre avec les valeurs du sous-jacent (`r=0.015, q=0, sigma=0.2, n_steps=8`):

```
[[100.         107.32706603 115.19099102 123.63111098 132.68964411 142.41190195 152.84651603 164.04568118 176.06541655]
 [  0.          93.17314234 100.         107.32706603 115.19099102 123.63111098 132.68964411 142.41190195 152.84651603]
 [  0.           0.          86.81234454  93.17314234 100.         107.32706603 115.19099102 123.63111098 132.68964411]
 [  0.           0.           0.          80.88578935  86.81234454  93.17314234 100.         107.32706603 115.19099102]
 [  0.           0.           0.           0.          75.36383164  80.88578935  86.81234454  93.17314234 100.        ]
 [  0.           0.           0.           0.           0.          70.21885013  75.36383164  80.88578935  86.81234454]
 [  0.           0.           0.           0.           0.           0.          65.42510919  70.21885013  75.36383164]
 [  0.           0.           0.           0.           0.           0.           0.          60.95863011  65.42510919]
 [  0.           0.           0.           0.           0.           0.           0.           0.          56.7970712 ]]
```



## Fonction de pricing dans l'arbre

On définit la fonction `tree_price` permettant d'évaluer un payoff quelconque dans l'arbre.

La fonction `payoff` fournie à `tree_price` s'évalue à partir du time step `i = 0 ... n_steps` où on se trouve et de la valeur de l'option aux `n_nodes = i+1` noeuds correspondants.

In [ ]:
# tree price as a function of option payoff
def tree_price (payoff) :
  # option value
  v = np.zeros(n_steps+1)
  v = payoff(n_steps, v) # payoff at final date T

  # discount between 2 time steps
  discount = math.exp(-r*dt)

	# backward loop
  for i in range(n_steps-1, -1, -1) : # => i = n_steps-1 ... 0
    n_nodes = i+1 # i+1 nodes at time step #i
    v[:n_nodes] = discount * ( p * v[:n_nodes] + (1-p) * v[1:n_nodes+1] )
    v[:n_nodes] = payoff(i, v[:n_nodes])

  return v[0]

## Pricing des options

On s'intéresse à l'évaluation :

* d'un call européen et du call américain correspondant
* d'un put européen et du put américain correspondant
* et en bonus, juste pour le fun, d'un call barrière down & out

Il suffit donc de définir les 5 payoffs correspondants et d'appeler la fonction `tree_price`.

On définit les fonctions `bs_call` et `bs_put` pour comparer les prix obtenus avec les prix Black & Scholes.

In [ ]:
# === option payoffs
K = 100 # strike
H = 90 # down & out call barrier

def european_call_payoff(i, v_i):
	if (i==n_steps):
		v_i = np.maximum(S[:,n_steps] - K, 0.)
	return v_i

def american_call_payoff(i, v_i):
	n_nodes = i+1 # i+1 nodes at time step #i
	return np.maximum(S[:n_nodes,i] - K, v_i)

def european_put_payoff(i, v_i):
	if (i==n_steps):
		v_i = np.maximum(K - S[:,n_steps], 0.)
	return v_i

def american_put_payoff(i, v_i):
	n_nodes = i+1 # i+1 nodes at time step #i
	return np.maximum(K - S[:n_nodes,i], v_i)

def down_and_out_call_payoff(i, v_i):
	if (i==n_steps):
		v_i = np.maximum(S[:,n_steps] - K, 0.)
	n_nodes = i+1 # i+1 nodes at time step #i
	v_i = np.where(S[:n_nodes,i] > H, v_i, 0.)
	return v_i

# tree price european & american calls & puts + down & out call
european_call = tree_price(european_call_payoff)
american_call = tree_price(american_call_payoff)
european_put  = tree_price(european_put_payoff)
american_put  = tree_price(american_put_payoff)
down_and_out_call = tree_price(down_and_out_call_payoff)


# BS functions
from scipy.stats import norm

# BS formula for a call
def bs_call(T, K, S0, sigma, r) :
	if (T==0 or sigma==0) : return max(S0-K, 0)
	sigma_sqrt_T = sigma * math.sqrt(T)
	d1 = (math.log(S0/K) + (r + 0.5 * sigma**2) * T) / sigma_sqrt_T
	d2 = d1 - sigma_sqrt_T
	return S0 * norm.cdf(d1) - K * math.exp(-r*T) * norm.cdf(d2)

# BS formula for a put
def bs_put(T, K, S0, sigma, r) :
	if (T==0 or sigma==0) : return max(S0-K, 0)
	sigma_sqrt_T = sigma * math.sqrt(T)
	d1 = (math.log(S0/K) + (r + 0.5 * sigma**2) * T) / sigma_sqrt_T
	d2 = d1 - sigma_sqrt_T
	return -S0 * norm.cdf(-d1) + K * math.exp(-r*T) * norm.cdf(-d2)

# display
F0 = S0*math.exp((r-q)*T)
df = math.exp(-r*T)
print("European call = {:.4f} (closed-form BS = {:.4f})".format(european_call , df * bs_call(T, K, F0, sigma, 0) ))
print("American call = {:.4f}".format(american_call))
print("European put  = {:.4f} (closed-form BS = {:.4f})".format(european_put, df * bs_put(T, K, F0, sigma, 0) ))
print("American put  = {:.4f}".format(american_put))
print("Down & out call = {:.4f}".format(down_and_out_call))


European call = 8.6702 (closed-form BS = 8.6728)
American call = 8.6702
European put  = 7.1814 (closed-form BS = 7.1840)
American put  = 7.3056
Down & out call = 7.2119


On constate que si $r>0$, en l'absence de dividendes ($q=0$), le prix de call américain est rigoureusement égal au prix du call européen.

En effet, à toute date $t<T$ le prix du call américain et supérieur ou égal au prix du call européen, lui même strictement plus cher que sa valeur intrinsèque :

$$
C_t^\text{us} \geq C_t^\text{eur} > \text{VI}_t = \left(S_t - Ke^{-r(T-t)}\right)^+
$$

Or, si $r>0$, on a $\text{VI}_t > S_t-K$ si bien que :

$$
C_t^\text{us} > S_t - K
$$

Autrement dit, à toute date $t<T$, il n'est jamais optimal d'exercer le call américain, puisque son exercice immédiat rapporte toujours moins que le fait de conserver l'option ! Et un call américain sans exercice anticipé est un call... européen !

En revanche, dans cette situation où $r>0$, le put américain vaut strictement plus cher que le put européen.

A l'inverse, si $r<0$, le call américain sera strictement plus cher que le call européen et le put américain aura strictement le même prix que le put européen car :

$$
P_t^\text{us} \geq P_t^\text{eur} > \text{VI}_t = \left(Ke^{-r(T-t)}- S_t\right)^+ > K-S_t
$$

A noter que si $r=0$ et $q=0$, les prix de nos 4 options (calls et puts américains et européens) coïncident parfaitement.

Dès qu'il y a des dividendes ($q>0$) on constate la chose suivante :

- Si $r>0$, et le call américain ET le put américain sont plus chers que leurs versions européenes.

- Si $r < 0$, le call américain est plus cher que le call européen, MAIS  le put américain a même prix que le put européen.

Pourquoi les prix des puts américain et européens coïncident-ils dans le cas $q>0$ ? Car puisque $Ke^{-r(T-t)}>K$ et $S_te^{-q(T-t)}<S_t$ on a :

$$
P_t^\text{us} \geq P_t^\text{eur} > \text{VI}_t = \left(Ke^{-r(T-t)}- S_te^{-q(T-t)}\right)^+ > K-S_t
$$

et la valeur d'un exercice immédiat du put américain rapporte toujours moins que la conservation de l'option. Il n'est jamais optimal de l'exercer de façon anticipée et il se transforme ainsi en put européen.

**Important** : on utilise ici le modèle de Black & Scholes (vers lequel converge l'arbre binomial quand on fait tendre le nombre de pas de temps vers l'infini) mais **les propriétés ci-dessus sur les prix des américaines sont complétement d'indépendantes du modèle utilisé**.
